In [32]:
from collections.abc import Mapping

class TransitionError(Exception):
    """
    Error raised during invalid transitions.
    """


class DFA:
    """
    Representa um Autömato Determinístico Finito.
    """
    
    def __init__(self, start_state, accept_states, transition=None):
        self.start_state = start_state
        self.accept_states = set(accept_states)
        
        if transition is None:
            transition = self.delta
        if callable(transition):
            self.delta = transition
        elif isinstance(transition, Mapping):
            def delta(state, symb): 
                try:
                    return transition[state, symb]
                except KeyError:
                    raise TransitionError(f'No transition defined for transition: {state}, {symb}')
                    
            self.delta = delta
            
    def run_verbose(self, src):
        """
        Run DFA and display its path on state space.
        """
        state = self.start_state
        delta = self.delta
        print('DFA:', state, end='')
        
        for symb in src:
            try:
                state = delta(state, symb)
                print(' ->', state, flush=True, end='')
            except TransitionError:
                print(' -> *error*')
                return None
        
        print()
        return state
    
    def run(self, src):
        """
        Run DFA and return final state.
        
        Return None if an invalid transition is found.
        """
        state = self.start_state
        delta = self.delta
        
        for symb in src:
            try:
                state = delta(state, symb)
            except TransitionError:
                return None
        return state
    
    def accept(self, src):
        """
        Checks if DFA accepts input.
        """
        state = self.start_state
        delta = self.delta
        try:
            for symb in src:
                state = delta(state, symb)
        except TransitionError:
            return False
        return state in self.accept_states

In [33]:
dfa = DFA('a', {'c'}, {('a', 'A'): 'b', ('b', 'A'): 'c', ('c', 'A'): 'c'})
dfa.accept('AAB')

False

In [61]:
class NFA:
    """
    Representa um Autömato Determinístico Finito.
    """
    
    def __init__(self, start_state, accept_states, transition=None):
        self.start_state = start_state
        self.accept_states = set(accept_states)
        
        if transition is None:
            transition = self.delta
        if callable(transition):
            self.delta = transition
        elif isinstance(transition, Mapping):
            def delta(state, symb): 
                try:
                    return transition[state, symb]
                except KeyError:
                    raise TransitionError(f'No transition defined for transition: {state}, {symb}')
                    
            self.delta = delta
            
    def run_verbose(self, src, print=print):
        """
        Run NFA and display its path on state space.
        """
        states = {self.start_state}
        delta = self.delta
        print('NFA:', states, end='')
        
        for symb in src:
            new_states = set()
            for st in states:
                try:
                    new_states.update(delta(st, symb))
                except TransitionError:
                    pass
            
            # Epsilon transitions
            epsilon = list(new_states)
            while epsilon:
                st = epsilon.pop()
                try:
                    for new in delta(st, None):
                        if new not in new_states:
                            new_states.add(new)
                            epsilon.append(new)
                except TransitionError:
                    pass
                    
            states = new_states
            print(' -> ', states, end='')
        print()
        return states
    
    def run(self, src):
        """
        Run NFA and return final states.
        
        Return None if an invalid transition is found.
        """
        return self.run_verbose(src, lambda *args, **kwargs: None)
    
    def accept(self, src):
        """
        Checks if DFA accepts input.
        """
        return bool(self.accept_states.intersection(self.run(src)))

In [62]:
nfa = NFA('a', {'c'}, {
    ('a', 'A'): {'b'}, 
    ('b', 'A'): {'c'}, 
    ('b', None): {'d'}, 
    ('c', 'A'): {'c'},
})
nfa.accept('AAB')

False

In [63]:
nfa.run_verbose('AAA')

NFA: {'a'} ->  {'d', 'b'} ->  {'c'} ->  {'c'}


{'c'}